In [68]:
import os
import pandas as pd
from collections import Counter

In [2]:
metadata = pd.read_excel("./data/COH DechArch Metadata.xlsx", sheetname="Italian",skiprows=[1])

In [3]:
metadata.columns

Index(['Folder', 'Filename', 'Nome foto', 'Anno', 'Luogo', 'Nome monumento',
       'Descrizione', 'Nome autore'],
      dtype='object')

In [4]:
metadata.head(5)

,Folder,Filename,Nome foto,Anno,Luogo,Nome monumento,Descrizione,Nome autore
0,1_SIRIA - Palmira,2) Palmira. T.di bel porticato ovest. 2' marzo...,Palmira _Tempio di Bel,1993,Palmira,Tempio di Bel,"Tempio di Bel, porticato ovest",DecArch_Marina Milella
1,1_SIRIA - Palmira,3) Palmira. Temenos T. di bel ingresso facciat...,Palmira _Tempio di Bel,1993,Palmira,Tempio di Bel,Tempio di Bel. Facciata interna dell'ingresso ...,DecArch_Marina Milella
2,1_SIRIA - Palmira,4) Palmira. Temenos T. di bel ingresso facciat...,Palmira _Tempio di Bel,1993,Palmira,Tempio di Bel,Tempio di Bel. Facciata interna dell'ingresso ...,DecArch_Marina Milella
3,1_SIRIA - Palmira,5) Palmira. Temenos T. di bel ingresso facciat...,Palmira _Tempio di Bel,1993,Palmira,Tempio di Bel,Tempio di Bel. Facciata interna dell'ingresso ...,DecArch_Marina Milella
4,1_SIRIA - Palmira,6) Palmira. Temenos T. di bel ingresso facciat...,Palmira _Tempio di Bel,1993,Palmira,Tempio di Bel,Tempio di Bel. Facciata interna dell'ingresso ...,DecArch_Marina Milella


# Luogo keywords

In [78]:
cnt = Counter(metadata.Luogo.tolist())
for token in all_tokens:
    cnt[token] += 1
cnt.most_common(30)

[('Palmira', 486),
 ('Damasco', 158),
 ('Aleppo', 110),
 ('Apamea', 108),
 ('Sergiopolis', 90),
 ('Bosra', 52),
 ('Zenobia', 22),
 ('Ebla', 8),
 ('Dura Europos', 8),
 ('Europos', 8),
 ('Dura', 8),
 ('Mari', 6),
 ('Ain Dara', 4),
 ('Ain', 4),
 ('Philippopoli', 4),
 ('Dara', 4)]

In [81]:
header = "== '''Luogo''' (place) ==\n"
header_row = """{| class="wikitable sortable" style="width: 60%; height: 200px;"
! Luogo
! frequency
! category
! wikidata
|-\n"""

data_rows = []
for l, count in cnt.most_common(1000):
    luogo = "| " + l + "\n"
    
    freq = "| " + str(count) + "\n"
    the_rest = "| \n| \n|-"
        
    row = luogo + freq + the_rest
    
    data_rows.append(row)
        
table_ending = "\n|}"
#print(data_rows)
luogo_wikitable = header + header_row + "\n".join(data_rows) + table_ending
print(luogo_wikitable)

== '''Luogo''' (place) ==
{| class="wikitable sortable" style="width: 60%; height: 200px;"
! Luogo
! frequency
! category
! wikidata
|-
| Palmira
| 486
| 
| 
|-
| Damasco
| 158
| 
| 
|-
| Aleppo
| 110
| 
| 
|-
| Apamea
| 108
| 
| 
|-
| Sergiopolis
| 90
| 
| 
|-
| Bosra
| 52
| 
| 
|-
| Zenobia
| 22
| 
| 
|-
| Ebla
| 8
| 
| 
|-
| Dura Europos
| 8
| 
| 
|-
| Europos
| 8
| 
| 
|-
| Dura
| 8
| 
| 
|-
| Mari
| 6
| 
| 
|-
| Ain Dara
| 4
| 
| 
|-
| Ain
| 4
| 
| 
|-
| Philippopoli
| 4
| 
| 
|-
| Dara
| 4
| 
| 
|-
|}


# Special place (Nome monumento + Luogo)

In [31]:
all_comb = zip(list(metadata["Nome monumento"]), list(metadata["Luogo"]))
all_comb = list(all_comb)
all_comb[5:10]

[('Tempio di Bel', 'Palmira'),
 ('Tempio di Bel', 'Palmira'),
 ('Palmira', 'Palmira'),
 ('Palmira', 'Palmira'),
 ('Tempio di Bel', 'Palmira')]

In [34]:
special = Counter(all_comb)
special.most_common(5)

[(('Museo', 'Damasco'), 46),
 ((nan, 'Apamea'), 42),
 (('Santuario S. Simeone', 'Aleppo'), 37),
 (('Grande Moschea', 'Damasco'), 24),
 (('Tempio di Bel', 'Palmira'), 21)]

In [67]:
header = "== Specific places (combination of '''Nome Monumento''' and '''Luogo''') ==\n"
header_row = """{| class="wikitable sortable" style="width: 60%; height: 200px;"
! Nome_monumento
! Luogo
! frequency
! category
! wikidata
|-\n"""


# make sure the values are sorted when printed, since wikitable syntax doesn't allow pre-sorted field
# http://stackoverflow.com/questions/16140758/default-sort-column-in-wikipedia-table
df = pd.DataFrame()
for *tup, cnt in special.items():
    for n, l in tup:
        row_data = pd.DataFrame([{"nome":n, "luogo":l, "freq":cnt}])
    df = df.append(row_data,ignore_index=True)

df.sort_values(by="freq", ascending=False, inplace=True)  
#print(df.head())

# Now create strings to form wikitable rows in sorted order
data_rows = []
for index, row in df.iterrows():
    nome = "| " + str(row["nome"]) + "\n"
    luogo = "| " + str(row["luogo"]) + "\n"
    freq = "| " + str(row["freq"]) + "\n"
    the_rest = "| \n| \n|-"
        
    new_row = nome + luogo + freq + the_rest
    
    if pd.notnull(row["nome"]) and pd.notnull(row["luogo"]) and row["nome"] != row["luogo"]:
        data_rows.append(new_row)
    else:
        continue
        
table_ending = "\n|}"
#print(data_rows)
special_wikitable = header + header_row + "\n".join(data_rows) + table_ending
print(special_wikitable)

== Specific places (combination of '''Nome Monumento''' and '''Luogo''') ==
{| class="wikitable sortable" style="width: 60%; height: 200px;"
! Nome_monumento
! Luogo
! frequency
! category
! wikidata
|-
| Museo
| Damasco
| 46
| 
| 
|-
| Santuario S. Simeone
| Aleppo
| 37
| 
| 
|-
| Grande Moschea
| Damasco
| 24
| 
| 
|-
| Tempio di Bel
| Palmira
| 21
| 
| 
|-
| Campo di Diocleziano
| Palmira
| 21
| 
| 
|-
| Museo
| Aleppo
| 18
| 
| 
|-
| Arco monumentale
| Palmira
| 18
| 
| 
|-
| Tempio funerario in rovina
| Palmira
| 17
| 
| 
|-
| Martirion
| Sergiopolis
| 16
| 
| 
|-
| Museo
| Palmira
| 15
| 
| 
|-
| Via colonnata
| Palmira
| 14
| 
| 
|-
| Santuario di Baal Shamin
| Palmira
| 12
| 
| 
|-
| Antiquarium
| Apamea
| 11
| 
| 
|-
| Terme di Diocleziano
| Palmira
| 9
| 
| 
|-
| Agorà
| Palmira
| 9
| 
| 
|-
| Tetrapilo
| Palmira
| 8
| 
| 
|-
| Cattedrale
| Sergiopolis
| 8
| 
| 
|-
| Tempio funerario
| Palmira
| 7
| 
| 
|-
| Basilica centrale
| Sergiopolis
| 7
| 
| 
|-
| Porta Nord
| Sergiopo